# Assignment 1

## Part 1: Using web-scraping to gather data

We wish to find the list of researchers that have joined the most important scientific conference in Computational Social Science in 2019.

We find them by webscraping the programmes of the 2019 edition via the flowwing links:  
>Oral presentations: https://2019.ic2s2.org/oral-presentations/    
> Poster presentations: https://2019.ic2s2.org/posters/    

Summary:
>We found 888 unique researchers in 2019
>Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?
- We took the decision to include authors that was listed as "no presentation" since we figured that we were looking for who were social data scientists and thus a cancelled presenter would still be valid.
- Futheremore, we decided to do an error search for names longer than 25 characters which helped us find about 10-15 more authors who would have ha

In [7]:
from bs4 import BeautifulSoup ##A package to work with HTML data
import requests #A package to make HTTP requests
import re

In [8]:
#Part 1: webscraping
#Oral Presenters, setup link for html webscraping
LINK = "https://2019.ic2s2.org/oral-presentations/"
r_oral = requests.get(LINK)
soup_oral = BeautifulSoup(r_oral.content)

In [9]:
#Extract all oral presenters form url
data = soup_oral.findAll("p") #extract the part with the relevant data
presenters_all_topics = []
for topics in data[3:]: #loop over all the topic sections
    presenters = []
    for presentation in topics: #loop over each presentation in each topic
        try:
            presenters_NoClock = presentation.split(" – ")[2] #get the string of names after each time slot
            presenters.append(presenters_NoClock)
        except:
            pass
        
    prenters_topic = (", ".join([lecture.split(".")[0] for lecture in presenters])).replace("<","").replace("–","") #only include the names of the presenters and not the topic
    prenters_topic = prenters_topic.split(", ") #turn into single list with all the names in the rest of the lines
    presenters_all_topics.append(prenters_topic)
    oral_presenters_2019 = [item for sublist in presenters_all_topics for item in sublist]

In [10]:
#Poster Presenters
#Setup link for html webscraping
LINK = "https://2019.ic2s2.org/posters/"
r_poster = requests.get(LINK)
soup_post = BeautifulSoup(r_poster.content)

In [11]:
table = soup_post.find("div",{"class":"col-md-8 page-content-wrap"}) #Get the section that contains all the data. We call it a table

In [12]:
poster_researchers_all = []
for day in table.findAll("ul"): #loop over both days
    for poster in day.findAll("li"): #loop over each poster  presentation
        if str(poster)[:12] != "<li><strong>": #from each poster presentation extract only the names and not the topic
            poster_researh_grouped = str(poster).split("<span>")[0][4:].replace("</li>","") #reformat the names to fit in a single list and exclude commas and "and"
            poster_researchers = [item for sublist in [team.split(" and ") for team in poster_researh_grouped.split(", ")] for item in sublist]
            poster_researchers_all.extend(poster_researchers)

In [13]:
all_presenters = poster_researchers_all + oral_presenters_2019
all_presenters = list(set(poster_researchers_all + oral_presenters_2019))
for i,presenter in enumerate(all_presenters): #Some semi-manual error detection for names over a given lenght. Unhash to perform a few error corrections due to inconsistency in typing of the website
    if len(presenter) > 25:
        print(i,presenter)        

list_remove = ['Keyword-Based Procedure for Identifying Research-Relevant Text</strong>','Abhishek Samantray',
               'Massimo Riccaboni','(Moved to 3D Text Analysis) Ivan Smirnov','No presentation: Vincent Antonio Traag',
               'Christos Nicolaides,Luis Cueto-Felgueroso', 'No presentation: Eelke Heemskerk', 'No presentation (cancelled)',
               'No presentation: Youyou Wu','Abhishek Samantray,Massimo Riccaboni', 'Hartmut Wessler,Scott Althaus']
for i in range(5):
    for name in list_remove:
        try:
            all_presenters.remove(name)
        except:
            pass
all_presenters.extend(['Abhishek Samantray','Massimo Riccaboni'])
all_presenters.extend(['Ivan Smirnov','Vincent Antonio Traag'])
all_presenters.extend(['Christos Nicolaides','Luis Cueto-Felgueroso','Abhishek Samantray','Massimo Riccaboni'])
all_presenters.extend(['Eelke Heemskerk', 'Hartmut Wessler','Scott Althaus'])
all_presenters.extend(['Rens Wilderom'])
all_presenters.extend('Youyou Wu')
all_presenters = sorted(all_presenters)[6:]
all_presenters

41 No presentation: Youyou Wu
69 (Moved to 3D Text Analysis) Ivan Smirnov
147 Hartmut Wessler,Scott Althaus
233 No presentation: Vincent Antonio Traag
237 and Rens Wilderom<br/>
<strong>Separating the Wheat from the Chaff: A Topic-
275 Mariken A.C.G. van der Velden
413 Gianmarco De Francisci Morales
472 Diego Fregolente Mendes de Oliveira
543 Abhishek Samantray,Massimo Riccaboni
646 No presentation (cancelled)
753 No presentation: Eelke Heemskerk
770 Zachary Steinert-Threlkeld
804 Christos Nicolaides,Luis Cueto-Felgueroso
875 Keyword-Based Procedure for Identifying Research-Relevant Text</strong>


['Aamena Alshamsi',
 'Abdullah Almaatouq',
 'Abe Hofman',
 'Abeer Aldayel',
 'Abhijnan Chakraborty',
 'Abhishek Samantray',
 'Abhishek Samantray',
 'Abigail Horn',
 'Abigail Jacobs',
 'Abu Sayeed Mondol',
 'Adam Hughes',
 'Adam Pah',
 'Adam Pah and Brian Uzzi',
 'Adina Nerghes',
 'Adrian Weller',
 'Adriana Iamnitchi',
 'Adrien Benamira',
 'Ahmad Alabdulkareem',
 'Akira Ishii',
 'Albert Laszlo Barabasi',
 'Alberto Antonioni',
 'Alberto Sánchez-Carralero',
 'Alejandro Espinosa-Rada',
 'Alejandro Noriega Campero',
 'Aleksandra Aloric',
 'Aleksandra Nenko',
 'Aleksandra Urman',
 'Aleksei Rotmistrov',
 'Alessandra Urbinati',
 'Alessandro Cossard',
 'Alessandro Provetti',
 'Alessandro Rosina',
 'Alex Furman',
 'Alex Pentland',
 'Alex Rutherford',
 'Alex ‘Sandy’ Pentland',
 'Alexander Robertson',
 'Alexander Sachs',
 'Alexandra Olteanu',
 'Alexandra Pang',
 'Alexandra Schofield',
 'Alexandre Bovet',
 'Alexandre Leroux',
 'Alexandre P Francisco',
 'Alexandru-Ionut Babeanu',
 'Alfredo Morales',

In [14]:
#Export unique names to .txt file
import codecs
all_presenters = sorted(set(all_presenters)) #Convert to a set data type to only get unique names and sort so the 4 time slot error can be manually removed
with codecs.open('all_presenters_2019.txt', 'w', "utf-8") as f:
    f.write(str(all_presenters))

print(len(sorted(set(all_presenters))),"unique researcher found as presenters for the 2019 conference and uploaded to a txt file")

888 unique researcher found as presenters for the 2019 conference and uploaded to a txt file


## Part 2: Getting data from the Semantic Scholar API

> * Consider the list of author ids you have found in Week 2, Part 3, first excercise. For each author, use the Academic Graph API to find:
>    - their _aliases_
>    - their _name_
>    - their _papers_, where for each paper we want to retain: 
>        -  _title_ 
>        -  _abstract_ 
>        -  the _year_ of publication
>        -  the _externalIds_ (this is because there are universal identifiers for scientific works called DOI that we can use across platforms)
>        -  _s2FieldsOfStudy_ the fields of study
>        - _citationCount_ the number of times that this paper was cited    
> * Create three dataframe to store the data you have collected. 
>    
>    - **Author dataset:** in the author dataset, one raw is one unique author, and each row contains the following information: 
>        - *authorId*: (str) the id of the author
>        - *name*: (str) the name of the author
>        - *aliases*: (list) the aliases of the author
>        - *citationCount*: (int) the total number of citations received by an author
>        - *field*: (str) the _s2FieldsOfStudy_ that occurs most times across an author's papers (you should first obtain the *category* for each _s2FieldsOfStudy_)
>    - **Paper dataset:** in the paper dataset, one row is one unique paper, and each row contains the following information:
>        - *paperId*: (str) the id of the paper
>        - *title*: (str) the title of the paper
>        - *year*: (int) the year of publication
>        - *externalId.DOI:* (str) the DOI of the paper
>        - *citationCount*: (int) the number of citations
>        - *fields*: (list) the fields included in the paper (you should first obtain the *category* for each _s2FieldsOfStudy_)
>        - *authorIds:* (list) this is a list of *author Ids*, including all the authors of this paper that are in our author dataset
>    - **Paper abstract dataset:** in the paper abstract dataset, one row is one unique paper, and each row contains the following information: 
>        - *paperId*: (str) the id of the paper
>        - *abstract*: (str) the abstract of the paper    
>  (Note: we keep the abstract separate to keep the size of files more manageable)


1. Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not.
2. Share the code you have used to solve the exercise above.
3. How long is your final _Author_ dataframe? How long is your final _Paper_ dataframe? 

(**Note**: If you did not manage to get all the years or all the authors' collaborators, you can still follow the exercise. Just remember to clarify your starting point.)



In [15]:
from bs4 import BeautifulSoup ##A package to work with HTML data
import requests #A package to make HTTP requests
import re

In [16]:
import random
random.seed(8)

In [144]:
#Due to heavy API loading time: Randomly pick a subset of the data gathered in part 1 
presenters_sample= sorted(set(random.sample(all_presenters,200)))
presenters_sample

['Adriana Iamnitchi',
 'Alejandro Noriega Campero',
 'Alessandro Cossard',
 'Alex Pentland',
 'Alexander Robertson',
 'Alexandra Olteanu',
 'Alexandra Schofield',
 'Ali Faqeeh',
 'Alina Sirbu',
 'Amit Sharma',
 'Andrea Bonanomi',
 'Andreas Diekmann',
 'Andreia Sofia Teixeira',
 'Andrew Rosenberg',
 'Aniket Alam',
 'Anne Cornelia Kroon',
 'Anton Boichenko',
 'Antonio Arechar',
 'Antonio Lima',
 'Antonio Molins',
 'Anxo Sanchez',
 'Arseny Rasov',
 'Aurelie Nardy',
 'Ayush K. Rai',
 'Ben Thies',
 'Benjamin Meindl',
 'Bernard Koch',
 'Bernard Rimé',
 'Bernardo Garcia Bulle',
 'Bjarke Mørch Mønsted',
 'Bryan Bruns',
 'Camille Roth',
 'Carlos Rodriguez Sickert',
 'Caspar Chorus',
 'Cesar Hidalgo',
 'Chaoming Song',
 'Chengcheng Li',
 'Chris Janssen',
 'Chung-Hong Chan',
 'Clara Christner',
 'Cor Veenman',
 'Dafne van Kuppevelt',
 'Damian Trilling',
 'Dana Fisher',
 'Daniel C. Fehder',
 'Daniel Ciganda',
 'Daniela Gawehns',
 'Daniele Guariso',
 'Daniele Silvestro',
 'David Gefen',
 'David Mar

In [145]:
import requests

BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/search"

my_url = BASE_URL + VERSION + RESOURCE
print(my_url)

#define function to accesss api to get authorIDs
def accessAPI(author_name):
    params = {'query':author_name,
           "offset":0,
           "limit":100,
            "fields":"authorId"}
    r = requests.get(my_url, params=params)
    if bool(r.json()["data"]):
        return r.json()["data"][0]

https://api.semanticscholar.org/graph/v1/author/search


In [146]:
#create list with authorIDs of the unique list of contributors to the 2019 conference
authorID_list_dict = []  
import time

count=0
dict_authors_match = {}
for x in presenters_sample[:14]:
    try:
        count+=1
        print(count/len(presenters_sample))
        auth_id = accessAPI(x)
        authorID_list_dict.append(auth_id)
        dict_authors_match[x] = auth_id['authorId']
    except:
        dict_authors_match[x] = None
    time.sleep(201/100) # wait a 1/100 of 5 minutes to have <100 requests per 5 minutes    

0.005
0.01
0.015
0.02
0.025
0.03
0.035
0.04
0.045
0.05
0.055
0.06
0.065
0.07


In [101]:
authorID_list_dict
authorID_list = [d['authorId'] for d in authorID_list_dict if d is not None] #list with dictionaries to list with strings and remove None fields
print(authorID_list)

['2036463605', '145618126', '145462309', '3251859', '2109842138', '1411234229', '47993785', '120663060', '144007353', '1415110072', '2123091329']


In [106]:
dict_authors_match

{'Alberto Sánchez-Carralero': None,
 'Alex Furman': {'authorId': '2036463605'},
 'Alexandra Pang': None,
 'Alfredo Morales': {'authorId': '145618126'},
 'Ana Alonso-Curbelo': None,
 'Andrea Santoro': {'authorId': '145462309'},
 'Andreas Breiter': {'authorId': '3251859'},
 'Angel Sanchez': {'authorId': '2109842138'},
 'Antske Fokkens': {'authorId': '1411234229'},
 'Anupama Aggarwal': {'authorId': '47993785'},
 'Armin Mertens': {'authorId': '120663060'},
 'Avidit Acharya': {'authorId': '144007353'},
 'Benjamin Devillers': {'authorId': '1415110072'},
 'Bernardo Garcia Bulle': {'authorId': '2123091329'}}

In [103]:
#Define a function to get all Aliases, Citation Counts (CC) and papers from the API when giving an AuthorID name as input
def get_author_info(authorID):
    string = "https://api.semanticscholar.org/graph/v1/author/" + str(authorID) + "?fields=aliases,citationCount,papers.fieldsOfStudy"
    name_id = requests.get(string).json()
    #name_id = [d['aliases'] for d in name_id] #list with dictionaries to list with strings
    #name_id = [item for sublist in name_id for item in sublist] #flatten list
    return name_id

#Run Loop to get all Aliases and Citation Counts (CC) for each authorID
authors_info = []
count=0
for x in authorID_list[:5]:
    #time.sleep(301/100) # Seconds
    try:
        authors_info.append(get_author_info(x))
        count+=1
        print(count/len(authorID_list))
    except:
        pass
    time.sleep(101/100) # wait a 1/100 of 5 minutes to have <100 requests per 5 minutes    


#aliases_list = [item for sublist in aliases_list for item in sublist] #flatten lis=papers.titles

0.09090909090909091
0.18181818181818182
0.2727272727272727
0.36363636363636365
0.45454545454545453


In [104]:
import pandas as pd
authors_df = pd.DataFrame.from_records(authors_info,index=['1', '2','3','4','5'])
authors_df

,authorId,aliases,citationCount,papers
1,2036463605,"[A. Furman, Alex Furman]",1304,[{'paperId': 'dba9d2ffbc0de67d15980c269b2051e2...
2,145618126,"[Alfredo Morales, Alfredo Trinidad Morales]",2,[{'paperId': 'd93237315b6c9fbffec85362d27e49e3...
3,145462309,"[A. Santoro, Andrea Santoro, Antonio Santoro]",10,[{'paperId': '0c686270ede685d2d8bcf6f5f513f918...
4,3251859,"[A. Breiter, Andreas Breiter]",610,[{'paperId': '2d9242f5d34a0a052a379d54d6e0588f...
5,2109842138,"[Angel Sanchez, Angel Alvaro Sanchez]",3,[{'paperId': 'cb5a72f0e9a78c65719387e0d4260639...


In [105]:
from collections import Counter
for i in range(len(authors_df['papers'])):
    authors_df['papers'][i] = Counter([d['fieldsOfStudy'][0] for d in authors_df['papers'][i] if d['fieldsOfStudy'] is not None]).most_common(1)[0][0]
authors_df

C:\Users\Ejer\AppData\Local\Temp\ipykernel_16716\372527164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_df['papers'][i] = Counter([d['fieldsOfStudy'][0] for d in authors_df['papers'][i] if d['fieldsOfStudy'] is not None]).most_common(1)[0][0]


,authorId,aliases,citationCount,papers
1,2036463605,"[A. Furman, Alex Furman]",1304,Mathematics
2,145618126,"[Alfredo Morales, Alfredo Trinidad Morales]",2,History
3,145462309,"[A. Santoro, Andrea Santoro, Antonio Santoro]",10,Medicine
4,3251859,"[A. Breiter, Andreas Breiter]",610,Computer Science
5,2109842138,"[Angel Sanchez, Angel Alvaro Sanchez]",3,Geography


In [126]:
list(dict_authors_match.keys())

['Alberto Sánchez-Carralero',
 'Alex Furman',
 'Alexandra Pang',
 'Alfredo Morales',
 'Ana Alonso-Curbelo',
 'Andrea Santoro',
 'Andreas Breiter',
 'Angel Sanchez',
 'Antske Fokkens',
 'Anupama Aggarwal',
 'Armin Mertens',
 'Avidit Acharya',
 'Benjamin Devillers',
 'Bernardo Garcia Bulle']

In [140]:
#Generate dataframe from original lsit of authors (also those for whom the APi could not find an ID)
names = list(dict_authors_match.keys()) 
ids = list(dict_authors_match.values())
data_tuples = list(zip(names,ids))
name_df = pd.DataFrame(data_tuples, columns=['names','authorId'])

In [142]:
name_df.merge(authors_df, how='left', on='authorId')

,names,authorId,aliases,citationCount,papers
0,Alberto Sánchez-Carralero,None,NaN,NaN,NaN
1,Alex Furman,2036463605,"[A. Furman, Alex Furman]",1304.0,Mathematics
2,Alexandra Pang,None,NaN,NaN,NaN
3,Alfredo Morales,145618126,"[Alfredo Morales, Alfredo Trinidad Morales]",2.0,History
4,Ana Alonso-Curbelo,None,NaN,NaN,NaN
5,Andrea Santoro,145462309,"[A. Santoro, Andrea Santoro, Antonio Santoro]",10.0,Medicine
6,Andreas Breiter,3251859,"[A. Breiter, Andreas Breiter]",610.0,Computer Science
7,Angel Sanchez,2109842138,"[Angel Sanchez, Angel Alvaro Sanchez]",3.0,Geography
8,Antske Fokkens,1411234229,NaN,NaN,NaN
9,Anupama Aggarwal,47993785,NaN,NaN,NaN


In [128]:
aliases_list = [d[''] for d in aliases_list if d is not None] #list with dictionaries to list with strings and remove None fields
aliases_list = list(filter(lambda item: item is not None, aliases_list)) #remove None values
aliases_list = [item for sublist in aliases_list for item in sublist] #flatten
print(aliases_list)

TypeError: list indices must be integers or slices, not str

In [ ]:
#Define function to get the citation count for each author
def get_citationCount(authorID):
    string = "https://api.semanticscholar.org/graph/v1/author/" + str(authorID) + "?fields=citationCount"
    name_id = requests.get(string).json()
    #name_id = [d['citationCount'] for d in name_id] #list with dictionaries to list with strings
    #name_id = [item for sublist in name_id for item in sublist] #flatten list
    return name_id

#Loop to get the Citation count of each author based on author ID
cc_list = []
for x in authorID_list:
    #time.sleep(301/100) # Seconds
    try:
        cc_list.append(get_citationCount(x))
    except:
        pass
    time.sleep(101/100) # wait a 1/100 of 5 minutes to have <100 requests per 5 minutes   

## Part 3: Law of large numbers.

As we have discussed in the lecture, one impact of heavy tails is that sample averages can be poor estimators of the underlying mean of the distribution.
To understand this point better, recall [the Law of Large Numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers).  Consider a sample of IID variables $ X_1, \ldots, X_n $ from the same distribution $ F $ with finite expected value $ \mathbb E |X_i| =  \int x F(dx)  = \mu $.

According to the law, the mean of the sample $ \bar X_n := \frac{1}{n} \sum_{i=1}^n X_i $ satisfies
<a id='equation-lln-as2'></a>
$$
\bar X_n \to \mu \text{ as } n \to \infty 
$$

This basically tell us that if we have a large enough sample, the sample mean will converge to the population mean. 

The condition that $ \mathbb E | X_i | $ is finite holds in most cases but can fail if the distribution $ F $ is very heavy tailed. Further, even when $ \mathbb E | X_i | $ is finite, the variance of a heavy tailed distribution can be so large that the sample mean will converge very slowly to the population mean. We will look into this in the following exercise. 


> 1. Sample __N=10,000__ data points from a [Gaussian Distribution](https://en.wikipedia.org/wiki/Normal_distribution) with parameters $\mu = 0 $ and $\sigma = 4$, using the [`np.random.standard_normal()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html) function. Store your data in a numpy array $\mathbf{X}$. 
> 2. Create a figure. 
>    - Plot the distribution of the data in $\mathbf{X}$. 
> 3. Compute the cumulative average of $\mathbf{X}$ (you achieve this by computing $average(\{\mathbf{X}[0],..., \mathbf{X}[i-1]\})$ for each index $i \in [1, ..., N+1]$  ). Store the result in an array. 
> 4. In a similar way, compute the cumulative standard error of $\mathbf{X}$. __Note__: the standard error of a sample is defined as $ \sigma_{M} = \frac{\sigma}{\sqrt(n)} $, where $\sigma$ is the sample standard deviation and $n$ is the sample size. Store the result in an array. 
> 5. Compute the values of the distribution mean and median using the formulas you can find on the [Wikipedia page of the Gaussian Distribution](https://en.wikipedia.org/wiki/Normal_distribution)  
> 6. Create a figure. 
>     - Plot the cumulative average computed in point 3. as a line plot (where the x-axis represent the size of the sample considered, and the y-axis is the average).
>     - Add errorbars to each point in the graph with width equal to the standard error of the mean (the one you computed in point 4). 
>     - Add a horizontal line corresponding to the distribution mean (the one you found in point 5).
> 7.  Compute the cumulative median of $\mathbf{X}$ (you achieve this by computing $median(\{\mathbf{X}[0],..., \mathbf{X}[i-1]\})$ for each index $i \in [1, ..., N+1]$). Store the result in an array. 
> 8. Create a figure. 
>    - Plot the cumulative median computed in point 7. as a line plot (where the x-axis represent the size of the sample considered, and the y-axis is the average).
>    - Add a horizontal line corresponding to the distribution median (the one you found in point 5).
>    - _Optional:_ Add errorbars to your median line graph, with width equal to the standard error of the median. You can compute the standard error of the median [via bootstrapping](https://online.stat.psu.edu/stat500/book/export/html/619). 
> 9. Now sample __N = 10,000__ data points from a [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution) with parameters $x_m=1$ and $\alpha=0.5$ using the [`np.random.pareto()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.pareto.html) function, and store it in a numpy array. (_Optional:_ Write yourself the function to sample from a Pareto distribution using the [_Inverse Transform Sampling method_](https://en.wikipedia.org/wiki/Inverse_transform_sampling))
> 10. Repeat points 2 to 8 for the Pareto Distribution sample computed in point 9. 
> 11. Now sample __N = 10,000__ data points from a [Lognormal Distribution](https://en.wikipedia.org/wiki/Log-normal_distribution) with parameters $\mu=0$ and $\sigma=4$ using the [`np.random.standard_normal()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html) function, and store it in a numpy array. 
> 12. Repeat points 2 to 8 for the Lognormal Distribution sample computed in point 11. 
> 13. Now, consider the array collecting the citations of papers from 2009 you created in Exercise 3, point 1. First, compute the mean and median number of citations for this population. Then, extract a random sample of __N=10,000__ papers.  
> 14. Repeat points 2,3,4,6,7 and 8 above for the paper citation sample prepared in point 13. 

> Answer the following questions:
(__Hint__: I suggest you plot the graphs above multiple times for different random samples, to get a better understanding of what is going on)

>    -  Compare the evolution of the cumulative average for the Gaussian, Pareto and LogNormal distribution. What do you observe? Would you expect these results? Why?
>    - Compare the cumulative median vs the cumulative average for the three distributions. What do you observe? Can you draw any conclusions regarding which statistics (the mean or the median) is more usfeul in the different cases? 
>    - Consider the plots you made using the citation count data in point 14. What do you observe? What are the implications? 
>    - What do you think are the main take-home message of this exercise? 